# User Guide

## Overview

Importing `xarray_bounds` registers a {py:class}`~xarray.Dataset` accessor named `bnds`. The `bnds` accessor provides utilities focused around 3 key operations:

1. [Access](#access) for bounds lookup;
2. [Assign](#assign) for creating and/or assigning bounds;
3. [Drop](#drop) for removing bounds.

To explore these operations, let’s load a dataset that contains boundary coordinates.

In [ ]:
import xarray as xr
import xarray_bounds as xrb

xr.set_options(keep_attrs=True)

ds = xrb.datasets.simple_bounds
ds

(access)=
## Accessing bounds

The `bnds` accessor returns a {py:class}`~collections.abc.Mapping` of variable names to boundary coordinates.

In [ ]:
ds.bnds

The `bnds` accessor is loosely modelled on the {py:class}`xarray.Coordinates` object. One important difference is that the keys are the names of the coordinates the bounds describe and **not** the bounds themselves.

```{note}
Boundary variables are accessed by the names of the coordinates they describe and **not** by their own variable names.
```

In [ ]:
ds.bnds['time']

If you want to access the bounds by their variable names, you can just use {py:attr}`~xarray.Dataset.coords` like normal.

```python
ds.coords['time_bnds']
```

### Aliases 

The ``bnds`` mapping supports using key aliases for item access and checking key membership. The aliases can be any name understood by {py:mod}`cf_xarray`:

- variable name;
- CF standard name;
- CF axis name. 


In [ ]:
# Variable name
name = 'lat'

aliases = [
    # CF Standard name
    'latitude',
    # CF Axis name
    'Y',
]

for alias in aliases:
    assert alias in ds.bnds

for alias in aliases:
    # square bracket notation
    xr.testing.assert_identical(
        ds.bnds[alias],
        ds.bnds[name],
    )
    # .get() method
    xr.testing.assert_identical(
        ds.bnds.get(alias),
        ds.bnds.get(name),
    )


Aliases are ignored during iteration. Iterating over the mapping only traverses the canonical keys.

In [ ]:
for name in ds.bnds:
    print(name)

The same is true for `len`, which returns the true length of the object, ignoring aliases.

In [ ]:
len(ds.bnds)

(drop)=
## Dropping bounds

To drop an object's existing bounds, we can use the {py:meth}`~xarray.Dataset.bnds.drop_bounds` method. The method supports aliases, just like with mapping keys. 

In [ ]:
ds2 = ds.copy()
ds2 = ds2.bnds.drop_bounds('Y')
ds2.bnds

If you don't pass any arguments to {py:meth}`~xarray.Dataset.bnds.drop_bounds`, all available bounds will be dropped.

In [ ]:
ds2 = ds2.bnds.drop_bounds()
ds2.bnds

In addition to dropping the specified bounds, the method also removes appropriate CF metadata attributes.

In [ ]:
'bounds' in ds2.time.attrs

(assign)=
## Adding bounds

### Deriving bounds

For dimension coordinates with regular* indexes, we can use {py:meth}`~xarray.Dataset.bnds.infer_bounds()` to automagically add boundary variables. As usual, aliases are supported.

In [ ]:
ds2 = ds2.bnds.infer_bounds('time')
ds2.bnds['time']

{py:meth}`~xarray.Dataset.bnds.infer_bounds()` can infer bounds for {py:class}`pandas.DatetimeIndex` and numeric {py:class}`pandas.Index` indexes. For datetime indexes, the index *must* have an inferrable frequency, while numeric indexes are *assumed* to be equally spaced.

See the [API documentation]() for more examples.

To control how bounds are inferred, we can specify which side of the bounds the index labels and which side of the bounds interval is closed.

In [ ]:
ds2 = ds2.bnds.infer_bounds('Y', label='middle', closed='left')
ds2.bnds['lat']

The function tries to derive sensible defaults if the ``label`` and/or ``closed`` arguments are not provided.

### Custom bounds

To add existing bounds objects, we can use {py:meth}`~xarray.Dataset.bnds.assign_bounds()`. 

In [ ]:
ds2 = ds2.bnds.assign_bounds(X=ds.bnds['lon'])
ds2.bnds['lon']

This method will assign the bounds and add appropriate CF metadata attributes.

In [ ]:
ds2.lon.attrs['bounds']

## Configuration

`xarray_bounds` offers a small number of configuration options through {py:func}`~xarray_bounds.set_options`. Currently, you can:

1. Control the name of the vertex dimension: `bounds_dim`

You can set these options either globally:

In [ ]:
xrb.set_options(bounds_dim='bounds')

or locally using a context manager:

In [ ]:
with xrb.set_options(bounds_dim='bounds'):
    # ... do bounds operation here ...
    pass

## Core functions

Most of the core operations exposed by the accessor are also available as [utility functions](api). These functions are particularly useful if you want to create bounds for datasets that do not conform to the CF convention.